Online resources: https://stackedit.io/app# , https://github.com/DataScienceUB/introduction-datascience-python-book , https://latex.codecogs.com/eqneditor/editor.php 

In [1]:
import pandas as pd # tratamiento de datos
import numpy as np
from scipy.stats import norm,gaussian_kde
import matplotlib.pyplot as plt 
import math

# from IPython.core.display import display, HTML

#### 4.4) Contraste de hipótesis o test de hipótesis

Dar una medida sobre la variabilidad de nuestras estimaciones es una manera de producir proposiciones estadísticas sobre la población, pero no la única manera. R.A. Fisher (1890–1962) propuso una alternativa, conocida como test de hipótesis, que se basa en el concepto de **significancia estadística**.

Supongamos que un análisis más profundo de los accidentes en Barcelona arroja una diferencia entre 2010 y 2013. Por supuesto, la diferencia podría solo haber sido casual, ya que se estima la variabilidad de ambas. Pero también podría ser que las condiciones del tráfico fuesen muy diferentes y que ambas podrían considerarse poblaciones diferentes. La cuestión es: ¿son los efectos observados reales o no?

Técnicamente, la cuestión se traduce a: ¿fueron los efectos observados estadísticamente significativos? El proceso de averiguar la significancia estadística es lo que se conoce como **test de hipótesis**.


El proceso comienza con dos hipótesis contrapuestas que compiten entre sí:

- $H_0$: la media de los accidentes diarios de tráfico en 2010 y 2013 son **iguales**. Hay una sola población, con una sola media real, y 2010 y 2013 son dos muestras diferentes de la misma población. Esta es conocida como hipótesis nula (*null hypothesis*) y los efectos observados son casuales. 
- $H_A$: la media de los accidentes diarios de tráfico en 2010 y 2013 son **distintas**. Son dos poblaciones diferentes, y las muestras de 2010 y 2013 pertenecen a poblaciones distintas. Esta es conocida como hipótesis alternativa (*alternative hypothesis*) y representa el otro punto de vista: los efectos observados son reales.

La norma general del contraste de hipótesis frecuentista radica en que no descartamos $H_0$  (y no consideraremos $H_A$) ***a menos que el efecto observado sea inverosímil bajo $H_0$***.

<hr>

### 4.4.1) Contraste de hipótesis uysando intervalos de confianza

Podemos usar el concepto que representan los intervalos de confianza para mediar la probabilidad de una hipótesis. Usaremos el ejemplo de los accidentes diarios de tráfico en Barcelona en 2010 y 2013

In [2]:
data = pd.read_csv("ACCIDENTS_GU_BCN_2010.csv",encoding="iso8859_15") # hay que cambiar el encoding a iso8859_15
data['Fecha']=data['Dia de mes'].apply(lambda x: str(x))+"-"+data['Mes de any'].apply(lambda x: str(x))+"-2010"
data['Fecha']=pd.to_datetime(data['Fecha'],dayfirst=True)
en2010 = data['Fecha'].value_counts()
print("Media en 2010: {}".format(en2010.mean()))
      
data = pd.read_csv("ACCIDENTS_GU_BCN_2013.csv",encoding="iso8859_15") # hay que cambiar el encoding a iso8859_15
data['Fecha']=data['Dia de mes'].apply(lambda x: str(x))+"-"+data['Mes de any'].apply(lambda x: str(x))+"-2013"
data['Fecha']=pd.to_datetime(data['Fecha'],dayfirst=True)
en2013 = data['Fecha'].value_counts()
print("Media en 2013: {}".format(en2013.mean()))

Media en 2010: 24.81095890410959
Media en 2013: 25.90958904109589


<hr>

En principio parece que la media de accidentes de tráfico en Barcelona aumentó en 2013. Pero... ¿es ésto estadísticamente significativo? Calculemos el intervalo de confianza. 

In [3]:
ci2013=[en2013.mean()-en2013.std()/math.sqrt(len(en2013)), en2013.mean()+en2013.std()/math.sqrt(len(en2013))]
print ("Intervalo de confianza en 2013: [{},{}]".format(ci2013[0],ci2013[1]))

Intervalo de confianza en 2013: [25.43283752308793,26.386340559103854]


**Como el valor estimado en 2010 *NO* entra dentro del intervalo de confianza de 2013, no podemos descartar la hipótesis alternativa.** Es decir, no se puede decir que en 2013 los accidentes de tráfico en Barcelona *se incrementasen de forma casual*. **NOTA PROPIA:** y rechazamos la hipótesis nula, es decir, podemos afirmar que es improbable que la media simplemente varíe de forma casual. 

##### Interpretando los testeos de intervalos de confianza

El contraste de hipótesis se forma rechazando o no la ***hipótesis nula***. Es decir, no rechazamos $H_0$ a menos que tengamos una importante evidencia en contra. ¿Pero qué entendemos con una *importante evidencia*? 

Como norma general, en aquellos casos donde la hipótesis nula es realmente verdad, no queremos incorrectamente rechazarla más de un 5% de las veces. Esto se corresponde con un nivel de significancia de $\alpha=0.05$. En este caso, la correcta interpretación de nuestro es:

- Si usamos un intervalo de confianza al 95% para testear un problema donde la hipótesis nula es verdadera, cometeremos un error cuando el punto estimado es al menos 1.96 veces el error estándar desde el parámetro de la población. Esto ocurre un 5% de las veces (2.5% por cada extremo).

### 4.4.2) Contraste de hipótesis usando p-values

Una noción más avanzada de la significancia estadistica la desarrolló R.A. Fisher en los años 20, cuando buscaba un test para decidir
si la variación en el crecimiento de las cosechas se debía a alguna intervención o eran factores aleatorios que se escapaban al control
experimental.
    
Fisher asumió primero que el fertilizante no causaba ninguna diferencia (hipótesis nula) y calculó P, la probabilidad de que un crecimiento observado en un campo fertilizado ocurriera si el fertilizante no tuviese un efecto real. Esta probabilidad se llama ***p-value*** (p-valor).

El p-valor es la probabilidad de observar datos, al menos de forma tan favorable a la hipótesis alternativa como nuestro conjunto de datos actuales, si la hipótesis nula es verdadera.  Normalmente usamos un resumen estadístico de los datos para ayudarnos a calcular el p-valor y evaluar la hipótesis. 

Si P es menor que 0.05 (la probabilidad de que un golpe de suerte sea menos que 5%) el resultado se declara **Estadísticamente significativo**. Debe señalarse que esta elección es muy arbitraria y no debería tomarse como una verdad científica.

El objetivo del contraste de hipótesis clásico es responder la cuestión: "dado una muestra, y un efecto aparente... ¿cuál es la probabilidad de ver ese efecto por casualidad?" y a esa pregunta respondemos: 

- El primer paso es **cuantificar el tamaño del efecto aparente**, escogiendo un test estadístico. En nuestro caso , el efecto aparente es la diferencia entre las razones de los accidentes, así que una elección natural para el test estadístico puede ser la diferencia en las medias de los dos períodos. **NOTA PROPIA:** para evaluarlo haré el cambio de calcular la diferencia en valor absoluto.
- El segundo paso es definir una **hipótesis nula**, que es el modelo del sistema baso en la asunción de que el efecto aparente no es real. En nuestro ejemplo, la hipótesis nula es que no hay diferencias entre los dos períodos.
- El tercer paso es calcular un **p-valor**, que es la probabilidad de ver el efecto aparente asumiendo cierta la hipótesis nula. En nuestro ejemplo, calculamos la diferencia entre las medias y entonces calculamos la probabilidad de ver una diferencia tan grande o más grande bajo la hipótesis nula.
- El último paso es interpretar el resultado. Si el p-valor es bajo se dice que el efecto es estadísticamente significativo, lo que significa que es improbable que haya ocurrido por casualidad. Y por tanto inferimos que el efecto es probable que aparezca en la más grande población.

<hr>

Así que nuestro test estadístico queda:

In [4]:
n = len(en2013)
m = len(en2010)
p = en2013.mean() - en2010.mean() 
print("Tamaños en 2010 m={} y en 2013 n={}".format(m,n))
print("Diferencia de medias es p={:.4f}".format(p))

Tamaños en 2010 m=365 y en 2013 n=365
Diferencia de medias es p=1.0986


<hr>

Para poder estimar nuestro p-valor podemos hacer lo siguiente: 

1. Juntar las distribuciones, generar las muestras de tamaño n y calcular sus medias.
2. Contar cuantas muestras son mayores que las observadas.



In [5]:
n = len(en2013)
m = len(en2010)
p = en2013.mean() - en2010.mean()
print("Tamaños en 2010 m={} y en 2013 n={}".format(m,n))
print("Diferencia de medias es p={:.4f}".format(p))

# juntando distribuciones
juntas = np.concatenate([en2010,en2013])
# print(juntas)
np.random.shuffle(juntas) #barajando los datos

# generando muestras
import random
s = 1000
diff = []
for i in range(s):
    p1 = [random.choice(juntas) for _ in range(n)] # primera muestra. n es el tamaño de la muestra
    p2 = [random.choice(juntas) for _ in range(n)] # segunda muestra
    diff.append(np.mean(p1)-np.mean(p2)) # NOTA PROPIA: diferencia entre las medias en valor absoluto.

diff2 = np.array(diff) # convierte la lista diff  en un array diff2.
w1 = np.where(diff2>p) # subarray en el que la condición es mayor que el valor p calculado antes.
probabilidad = np.shape(w1)[1]/float(s)

print("El p-valor es {} , {}% donde p vale {:.5f}".format(probabilidad,100*probabilidad,p))

if (probabilidad<0.05):
    print("El efecto es probable, menor al 5%")
else: 
    print("El efecto es improbable, mayor al 5%")


Tamaños en 2010 m=365 y en 2013 n=365
Diferencia de medias es p=1.0986
El p-valor es 0.042 , 4.2% donde p vale 1.09863
El efecto es probable, menor al 5%


<hr><hr>

A ver, tengo dos grandes dudas
1. ¿No debería tomar la diferencia como valor absoluto? No, creo que no.
2. A ver, la hipótesis nula es que no hay diferencias entre ambas muestras ¿no? Si yo barajo y obtengo más muestras, la hipótesis nula se confirma cada vez que la diferencia sea menor que p. ¿O le ha dado la vuelta y la hipótesis nula es que aumentan las diferencias?

- Considero que las muestras originales de 2010 y 2013, en media, no van a aumentar de ese valor p calculado de referencia. Esa es la hipótesis nula.
- Creo muestras al azar con esos datos y calculo sus diferencias en medias. ¿Cuántas aumentan?
- La interpretación correcta es que esas diferencias de medias es <u>improbable que aumenten</u>, ya que no tienen significancia estadística. Ocurren raramente, menos del 5% de las veces. 

<hr><hr>

<hr>

**Otra explicación al p-valor. Pág:** [p-valor en datatab](https://datatab.es/tutorial/p-value) , [p-value in youtube](https://www.youtube.com/watch?v=Xni5YVlgiiQ), [p-value2 en youtube](https://youtu.be/ibmjzFv4K1Y?si=S63CXBcpoEGEVofr) y [p-value3 en Youtube](https://www.youtube.com/watch?v=kW7nrzLpZGo)

### ¿Qué es el valor p?
[p-value3 en Youtube](https://www.youtube.com/watch?v=kW7nrzLpZGo)

- **Hipótesis:** aseveración sobre una población
- **Prueba de hipótesis:** procedimiento basado en la evidencia de una muestra para probar la aseveración sobre una población.

#### Ejemplo ####
- Antes: hace 15 años el 27% de los árboles de un bosque contaba con cavidades que podían albergar nidos de aves
- Ahora: la cantidad de árboles que contienen cavidades ha aumentado.
- Tomamos la hiṕotesis nula como $H_0: p=0.27$, es decir ***aseveramos que no se produce ningún cambio, no hay ningún efecto apreciable***. (Falley 2019)
- Y la hipótesis que vamos a investigar es la hipótesis alternativa $H_A: p>0.27$, es decir una hipótesis contraria a $H_0$ indicando que se ha producido un cambio. (Falley 2019, PSU 2023)

#### Ejemplo 2

- Imagino que tengo un dado en el que quiero saber si está trucado para que salga un cinco.
- Planteo la ***hipótesis nula*** como $H_0: p(5) = 1/6$ La probabilidad de que cada vez que lo tiro salga un cinco.
- Planteo la ***hipótesis alternativa*** como $H_0: p(5) > 1/6$ La probabilidad de que cada vez que lo tiro salga un cinco mayor de 1/6.
- Busco una **evidencia muestral**. Por ejemplo, que en 20 tiradas salga 17 veces 5 y otras tres otra cosa. "5-5-5-5-5-5-5-5-5-5-5-5-5-5-5-5-5-3-2-6"
- Y ahora me planteo. **Tomando cierta la hipótesis nula** , ¿en qué condiciones de probabilidad obtengo esa muestra? Esa muestra en concreto con 17 veces el número 5 de 20 veces tiene una probabilidad de $P=4.032^{-11}$ muy pequeña. 
- Ese valor de probabilidad es el p-value, --> $p-value=4.032^{-11}$
- Y lo importante es la **significancia estadística**. Como ese valor *es menor* que $\alpha=0.05$ es muy poco significativo. Entonces esa evidencia tiende *a desacreditar la hipótesis nula y a respaldar la hipótesis alternativa* (Witte & Witte 2017).

### Definción de la American Statistical Association
- Un valor p es la probabilidad, bajo un model oestadístico específico, de que un valor observado de los datos sea igual o más extremo que el valor observado.
